In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import joblib
import tensorflow_graphics.nn.loss as tfg_loss
import tensorflow_probability as tfp
from keras.utils.np_utils import normalize
import os

SIZE_X = 112
SIZE_Y = 112

# Definisi fungsi-fungsi yang diperlukan
def dice_coef(y_true, y_pred, smooth=1.0):
    class_num = 4
    total_loss = 0
    for i in range(class_num):
        y_true_f = K.flatten(y_true[:,:,:,i])
        y_pred_f = K.flatten(y_pred[:,:,:,i])
        intersection = K.sum(y_true_f * y_pred_f)
        loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))
        total_loss += loss
    total_loss /= class_num
    return total_loss

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def hd95_metric(y_true, y_pred):
    value = tfg.nn.loss.hausdorff_distance.evaluate(y_true, y_pred)
    percentile = tfp.stats.percentile(value, 95.0, interpolation='lower')
    return percentile

# Path model yang disimpan
model_path = "C:/Users/Lab129/indisalsa/UNet_Tesis_250.model"
encoder_path = model_path + "_encoder.joblib"

# Memuat model
model = load_model(model_path, custom_objects={
    'dice_coef': dice_coef,
    'hd95_metric': hd95_metric,
    'precision': precision,
    'sensitivity': sensitivity,
    'specificity': specificity
})

# Memuat encoder
encoder = joblib.load(encoder_path)

# Fungsi untuk memproses gambar input
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE_X, SIZE_Y))
    image = np.expand_dims(image, axis=2)  # Menambah dimensi channel
    image = normalize(image, axis=1)  # Normalisasi gambar
    image = np.expand_dims(image, axis=0)  # Menambah dimensi batch
    return image

# Fungsi untuk memproses ground truth mask
def preprocess_mask(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (SIZE_X, SIZE_Y), interpolation=cv2.INTER_NEAREST)
    return mask

# Fungsi untuk memetakan ulang nilai prediksi
def remap_labels(mask):
    label_map = {0: 0, 1: 64, 2: 128, 3: 255}
    remapped_mask = np.zeros_like(mask)
    for key, value in label_map.items():
        remapped_mask[mask == key] = value
    return remapped_mask

# Fungsi untuk melakukan prediksi pada gambar
def predict_image(image_path):
    image = preprocess_image(image_path)
    pred_mask = model.predict(image)
    pred_mask = np.argmax(pred_mask, axis=-1)
    pred_mask = np.squeeze(pred_mask, axis=0)
    pred_mask = remap_labels(pred_mask)  # Pemetaan ulang nilai prediksi
    return pred_mask

# Fungsi untuk menampilkan gambar input, ground truth, dan hasil prediksi
def display_prediction(image_path, mask_path, pred_mask):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (SIZE_X, SIZE_Y))
    
    ground_truth = preprocess_mask(mask_path)
    
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.title("Input Image")
    plt.imshow(image, cmap='gray')

    plt.subplot(1, 3, 2)
    plt.title("Ground Truth Mask")
    plt.imshow(ground_truth, cmap='gray')

    plt.subplot(1, 3, 3)
    plt.title("Predicted Mask")
    plt.imshow(pred_mask, cmap='gray')

    plt.show()

# Path direktori gambar input
input_dir = "C:/Users/Lab129/Downloads/Indi/Dataset Seg 128"

# Path direktori untuk menyimpan hasil prediksi
output_dir = "C:/Users/Lab129/Downloads/Indi/HasilSeg"

# Membuat direktori output jika belum ada
os.makedirs(output_dir, exist_ok=True)

# Looping untuk semua folder
for folder_name in os.listdir(input_dir):
    if not os.path.isdir(os.path.join(input_dir, folder_name)):
        continue  # Skip jika bukan folder
    folder_path = os.path.join(input_dir, folder_name)
    output_folder_path = os.path.join(output_dir, folder_name)
    os.makedirs(output_folder_path, exist_ok=True)  # Membuat direktori output

    # Looping untuk semua gambar dalam folder
    for i in range(128):
        image_name = f"flair_{i:03d}.jpg"
        image_path = os.path.join(folder_path, "Images", image_name)
        mask_path = os.path.join(folder_path, "Masks", f"seg_{i:03d}.png")
        output_mask_path = os.path.join(output_folder_path, f"seg_{i:03d}.png")

        # Prediksi gambar
        pred_mask = predict_image(image_path)

        # Menyimpan hasil prediksi
        plt.imsave(output_mask_path, pred_mask, cmap='gray')

        print(f"Processed: {image_path}")

print("Prediction and saving completed.")

C:\Users\Lab129\AppData\Roaming\Python\Python310\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\Lab129\AppData\Roaming\Python\Python310\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not fil

Processed: C:/Users/Lab129/Downloads/Indi/Dataset Seg 128\BraTS20_Training_001\Images\flair_000.jpg
Processed: C:/Users/Lab129/Downloads/Indi/Dataset Seg 128\BraTS20_Training_001\Images\flair_001.jpg
Processed: C:/Users/Lab129/Downloads/Indi/Dataset Seg 128\BraTS20_Training_001\Images\flair_002.jpg
Processed: C:/Users/Lab129/Downloads/Indi/Dataset Seg 128\BraTS20_Training_001\Images\flair_003.jpg
Processed: C:/Users/Lab129/Downloads/Indi/Dataset Seg 128\BraTS20_Training_001\Images\flair_004.jpg
Processed: C:/Users/Lab129/Downloads/Indi/Dataset Seg 128\BraTS20_Training_001\Images\flair_005.jpg
Processed: C:/Users/Lab129/Downloads/Indi/Dataset Seg 128\BraTS20_Training_001\Images\flair_006.jpg
Processed: C:/Users/Lab129/Downloads/Indi/Dataset Seg 128\BraTS20_Training_001\Images\flair_007.jpg
Processed: C:/Users/Lab129/Downloads/Indi/Dataset Seg 128\BraTS20_Training_001\Images\flair_008.jpg
Processed: C:/Users/Lab129/Downloads/Indi/Dataset Seg 128\BraTS20_Training_001\Images\flair_009.jpg
